# Gerekli Kütüphaneler

In [7]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn import metrics

seed = 42
np.random.seed(seed)
random.seed(seed)

# Veri Seti

In [34]:
X, y = make_blobs(n_samples=100, centers=3, n_features=10, random_state=seed)

# Feature Agglomeration

In [35]:
class FeatureAgglomeration:
    def __init__(self, n_clusters=2):
        self.n_clusters = n_clusters
        self.labels_ = None

    def fit(self, X):
        n_features = X.shape[1]

        # Baslangicta her ozellilk bir kume olarak belirlenir
        clusters = {i: [i] for i in range(n_features)}
        self.labels_ = np.arange(n_features)

        # Ozellikler arasindaki mesafe hesaplanir
        distances = np.zeros((n_features, n_features))
        for i in range(n_features):
            for j in range(i + 1, n_features):
                distances[i, j] = np.linalg.norm(X[:, i] - X[:, j])
                distances[j, i] = distances[i, j]

        # Kumeleme islemi
        while len(clusters) > self.n_clusters:
            # Kucuk mesafe bulunur
            min_dist = np.inf
            min_pair = None

            for i in clusters:
                for j in clusters:
                    if i < j:
                        # Iki kume arasindaki ortalama mesafe hesaplanir
                        dist = np.mean([distances[p1, p2] for p1 in clusters[i] for p2 in clusters[i]])
                        if dist < min_dist:
                            min_dist = dist
                            min_pair = (i, j)

            # Kume ciftleri birlestirilir
            i, j = min_pair
            clusters[i].extend(clusters[j])
            del clusters[j]

            # Etiketler guncellenir
            for feature in clusters[i]:
                self.labels_[feature] = i

        return self

    def transform(self, X):
        X_reduced = np.zeros((X.shape[0], self.n_clusters))
        for i in range(self.n_clusters):
            X_reduced[:, i] = np.mean(X[:, self.labels_ == i], axis=1)

        return X_reduced

# Eğitim

In [36]:
fa = FeatureAgglomeration(n_clusters=2)
fa.fit(X)

# Sonuçlar

In [37]:
X_reduced = fa.transform(X)

In [38]:
print(X.shape)

(100, 10)


In [39]:
print(X_reduced.shape)

(100, 2)
